# Рекомендация тарифов

В данном проекте у нас есть данные по которым необходимо построить модель для задачи классификации, которая выберет подходящий тариф. 



<b><font size=5>Содержание</font></b><a name="to_content."></a>
* [Описание данных](#1.)
* [Задачи](#2.)
*  [Шаг 1. Открыть файл с данными и изучить его.](#3.)
     - [Вывод по первому шагу](#3.1.)
*  [Шаг 2. Разделение исходных данных на выборки.](#4.)
     - [Вывод по второму шагу](#4.1.)
* [Шаг 3. Исследование качества разных моделей.](#5.)
     - [Вывод по третьему шагу](#5.1.)
*  [Шаг 4. Проверка качество модели на тестовой выборке.](#6.)
     - [Вывод по четвертому шагу](#6.1.)     
* [Шаг 5. Проверка модели на вменяемость.](#7.)
     - [Общий вывод](#7.1.) 

<b><font size=5>Описание данных</font></b><a name="1."></a>
[<font size="2">(к содержанию)</font>](#to_content.)

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

* сalls — количество звонков,
* minutes — суммарная длительность звонков в минутах,
* messages — количество sms-сообщений,
* mb_used — израсходованный интернет-трафик в Мб,
* is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

<b><font size=5>Задачи</font></b><a name="2."></a>
[<font size="2">(к содержанию)</font>](#to_content.)

* Шаг 1. Открыть файл с данными и изучить его. 
* Шаг 2. Разделение исходных данных на обучающую, валидационную и тестовую выборки.
* Шаг 3. Исследование качества разных моделей.
* Шаг 4. Проверка качество модели на тестовой выборке.
* Шаг 5. Проверка модели на вменяемость.

<b><font size=5>Шаг 1. Открыть файл с данными и изучить его.</font></b><a name="3."></a>
[<font size="2">(к содержанию)</font>](#to_content.)

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('***')

In [3]:
df.sample(5)

,calls,minutes,messages,mb_used,is_ultra
3009,92.0,603.33,29.0,16178.64,0
924,27.0,195.40,0.0,15765.07,0
1132,51.0,381.12,43.0,14771.91,0
1140,40.0,195.79,55.0,19493.38,0
1128,13.0,118.15,6.0,4961.28,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [6]:
df.isnull().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

<b><font size=4>Вывод по первому шагу</font></b><a name="3.1"></a>
[<font size="2">(к содержанию)</font>](#to_content.)

В данных всё в порядке. Перейдём к следующему шагу.

<b><font size=5>Шаг 2. Разделение исходных данных на выборки.</font></b><a name="4."></a>
[<font size="2">(к содержанию)</font>](#to_content.)

Для начала посмотрим как распределены классы в нашей выборке.

In [7]:
df.is_ultra.value_counts(normalize=True)

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

Как видим, что 70% составляет тариф "Смарт" и 30% составляет тариф "Ультра". 

Теперь разобьём наши данные на 2 выборки с учётом распределения классов:
    
   * 1. Обучающаяся выборка, которая равна 75% от общих данных
   * 2. Тестовая выборка, которая равна 25% от общих данных

In [8]:
df_train, df_test = train_test_split(df, test_size=0.25, random_state=0, stratify=df.is_ultra)

Проверим, верно ли мы разбили наши данные

In [9]:
print(f'Размер обучающей выборки составляет: {df_train.shape[0] / len(df):.1%}')
print(f'Размер тестовой выборки составляет: {df_test.shape[0] / len(df):.1%}')

Размер обучающей выборки составляет: 75.0%
Размер тестовой выборки составляет: 25.0%


После того как наши выборки готовы, то создадим переменные для признаков и целевого признака для каждой выборки

In [10]:
features_train = df_train.drop('is_ultra', axis=1)
target_train = df_train['is_ultra']
features_test = df_test.drop('is_ultra', axis=1)
target_test = df_test['is_ultra']

<b><font size=4>Вывод по второму шагу</font></b><a name="4.1"></a>
[<font size="2">(к содержанию)</font>](#to_content.)

После того как мы разбили наши данные на соотношение 4:1(4 - тренировочная модель, 1 - тестовая выборка), а так же разбили на  признаки и целевого признаки, то перейдём к выбору модели и исследования качества выбранной модели.

<b><font size=5>Шаг 3. Исследование качества разных моделей.</font></b><a name="5."></a>
[<font size="2">(к содержанию)</font>](#to_content.)



Посмотрев на данные и задачу, которую нам предстоить решить, можно сделать вывод, что перед нами стоит задача классификации, а значит, что мы будем использовать простые модели, которые нам известны на данный момент:
   * Деревья Решений
   * Случайный лес
   * Логистическая регрессия
   
И выберем наилучшую из них. 

In [11]:
clf_dtc = DecisionTreeClassifier()
parametrs_dtc = {'max_depth': range (1,11)}
grid_dtc = GridSearchCV(clf_dtc, parametrs_dtc, cv=5, n_jobs=-1)
grid_dtc.fit(features_train, target_train)
print('Точность лучшей модели', grid_dtc.best_score_, 'с параметрами', grid_dtc.best_params_ )

Точность лучшей модели 0.7950207468879668 с параметрами {'max_depth': 3}


In [12]:
clf_rfc = RandomForestClassifier()
parametrs_rfc = {'n_estimators': range (10, 41, 10), 'max_depth': range (1,13)}
grid_rfc = GridSearchCV(clf_rfc, parametrs_rfc, cv=5, n_jobs=-1)
grid_rfc.fit(features_train, target_train)
print('Точность лучшей модели', grid_rfc.best_score_, 'с параметрами', grid_rfc.best_params_ )

Точность лучшей модели 0.8078838174273859 с параметрами {'max_depth': 9, 'n_estimators': 40}


In [13]:
clf = LogisticRegressionCV(Cs=10, cv=5, random_state=0).fit(features_train, target_train)
clf.scores_[1].mean(axis=0).max()

0.7510429364840333

<b><font size=4>Вывод по третьему шагу</font></b><a name="5.1"></a>
[<font size="2">(к содержанию)</font>](#to_content.)

Итак, мы исследовали 3 модели (Дерево решений, Случайный лес, Логистическая регрессия), которые показали:

* У дерева решений при глубине в 3, точность наилучшей модели на валидационной выборке равна: 0.79
* У случайного леса при 30 деревьях и глубине 8, точность наилучшей модели на валидационной выборке равна: 0.80
* У Логистической регрессии точность модели на валидационной выборке равна: 0.75


Исходя из этих данных можно сделать вывод, что лучшей моделью является Случайный лес при 40 деревьях с глубиной 8

<b><font size=5>Шаг 4. Проверка качество модели на тестовой выборке.</font></b><a name="6."></a>
[<font size="2">(к содержанию)</font>](#to_content.)

In [14]:
best_model = grid_rfc.best_estimator_
prediction = best_model.predict(features_test)
accuracy = accuracy_score(prediction, target_test)
print('Точность тестовой модели равна:', accuracy)

Точность тестовой модели равна: 0.8271144278606966


<b><font size=4>Вывод по четвертому шагу</font></b><a name="6.1"></a>
[<font size="2">(к содержанию)</font>](#to_content.)

Итак, точность тестовой модели составила 0.81, что больше 0.75

<b><font size=5>Шаг 5. Проверка модели на вменяемость.</font></b><a name="7."></a>
[<font size="2">(к содержанию)</font>](#to_content.)

Для того, чтобы оценить нашу модель на вменяемость, то посмотрим не предсказывает ли наша модель одни 0 или 1. Для этого глянем на общую долю того или оного тарифа. 

In [15]:
df.is_ultra.value_counts(normalize=True)

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

Как видим, что если бы наша модель предсказывала только 1 или 0, то результат бы её точности был равен либо 0.30, либо 0.69, следовательно наша модель предсказывает верно.

<b><font size=4>Общий вывод</font></b><a name="7.1"></a>
[<font size="2">(к содержанию)</font>](#to_content.)

Наилучшая модель была случайным лесом, которая показала точность тестовой модели в 0.81, а так же прошла проверку на вменяемость. 